# Parallel and Distributed Machine Learning

*The material in this notebook is based on the open-source content from [Dask's tutorial repository](https://github.com/dask/dask-tutorial). Why? Because they did it so well. Thank you, Dask contributors!*

We've now seen how Dask makes data analysis scalable with parallelization via Dask DataFrames. Let's now see how [Dask-ML](https://ml.dask.org/) allows us to do machine learning in a parallel and distributed manner. Note, machine learning is really just a special case of data analysis (one that automates analytical model building), so the 💪 Dask gains 💪 we've seen will apply here as well!

(If you'd like a refresher on the difference between parallel and distributed computing, [here's a good discussion on StackExchange](https://cs.stackexchange.com/questions/1580/distributed-vs-parallel-computing).)

In this notebook, we'll 

* break down machine learning scaling problems into two categories.
* review how Scikit-Learn works.
* solve an ML problem with a single machine (with Scikit-Learn).
* solve an ML problem with a single machine and parallelism (with Scikit-Learn and Joblib).
* solve an ML problem with a multiple machines and parallelism (with Scikit-Learn, Joblib, and Dask).
* solve an ML problem with larger-than-memory data using multiple machines and parallelism (Dask-ML).
* solve two ML problem with multiple machines *in the cloud* and parallelism (with Scikit-Learn, Joblib, Dask, and Coiled).


## Types of scaling problems in machine learning

So you have your machine learning workflow that works well for small problems. Then there are two main types of scaling challenges you can run into: scaling the **size of your data** and scaling the **size of your model**. That is:

1. **CPU-bound problems**: Data fits in RAM, but training takes too long. Many hyperparameter combinations, a large ensemble of many models, etc.
2. **Memory-bound problems**: Data is larger than RAM, and sampling isn't an option.

Here's a handy diagram for visualizing these problems:

<img src="images/dimensions_of_scale.svg" width="50%">

In the bottom-left quadrant, your datasets aren’t too large (and therefore fit comfortably in RAM) and your model isn’t too large. Here, you’re much better off using something like scikit-learn, XGBoost, and similar libraries. You don't need to leverage multiple machines in a distributed manner with a library like Dask-ML here.

If you’re in any of the other quadrants, however, distributed machine learning is the way to go.

Here's a bird's eye view of the strategy we'll apply in this notebook:

* For in-memory problems, just use scikit-learn (or your favorite ML library).
* For large models, use `dask_ml.joblib` and your favorite scikit-learn estimator.
* For large datasets, use `dask_ml` estimators.

## Scikit-Learn in five minutes

<img src="images/scikit_learn_logo_small.svg" alt="scikit-learn logo"/>


In this section, we'll quickly run through a typical Scikit-Learn workflow:

* Load some data (in this case, we'll generate it)
* Import the Scikit-Learn module for our chosen ML algorithm
* Create an estimator for that algorithm and fit it with our data
* Inspect the learned attributes
* Check the accurary of our model

Scikit-Learn has a nice, consistent API:

* You instantiate an `Estimator` (e.g. `LinearRegression`, `RandomForestClassifier`, etc.). All of the models *hyperparameters* (user-specified parameters, not the ones learned by the estimator) are passed to the estimator when it's created.
* You call `estimator.fit(X, y)` to train the estimator.
* Use `estimator` to inspect attributes, make predictions, etc. 

Here `X` is an array of *feature variables* (what you're using to predict) and `y` is an array of *target variables* (what we're trying to predict).

Here's the workflow. Let's first generate some random data.

In [1]:
from sklearn.datasets import make_classification

# Generate data
X, y = make_classification(n_samples=10000, n_features=4, random_state=0)

# View X
X[:8]

array([[-0.77244139,  0.3607576 , -2.38110133,  0.08757   ],
       [ 1.14946035,  0.62254594,  0.37302939,  0.45965795],
       [-1.90879217, -1.1602627 , -0.27364545, -0.82766028],
       [-0.77694695,  0.31434299, -2.26231851,  0.06339125],
       [-1.17047054,  0.02212382, -2.17376797, -0.13421976],
       [ 0.79010037,  0.68530624, -0.44740487,  0.44692959],
       [ 1.68616989,  1.6329131 , -1.42072654,  1.04050557],
       [-0.93912893, -1.02270838,  1.10093827, -0.63714432]])

*A quick [ML refresher](https://scikit-learn.org/stable/getting_started.html):*
- *`X` is the samples matrix (or design matrix). The size of `X` is typically (`n_samples`, `n_features`), which means that samples are represented as rows and features are represented as columns.*
- *A "feature" (also called an "attribute") is a measurable property of the phenomenon we're trying to analyze. A feature for a dataset of employees might be their hire date, for example.*

In [2]:
# View y
y[:8]

array([0, 0, 1, 0, 0, 0, 0, 1])

*A quick [ML refresher](https://scikit-learn.org/stable/getting_started.html):*
- *`y` are the target values, which are real numbers for regression tasks, or integers for classification (or any other discrete set of values). For unsupervized learning tasks, `y` does not need to be specified. `y` is usually 1d array where the `i`th entry corresponds to the target of the `i`th sample (row) of `X`.*

We'll fit a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) for this example, so let's load the appropriate Scikit-Learn module.

In [3]:
from sklearn.svm import SVC

Now we create the estimator and fit it.

In [4]:
estimator = SVC(random_state=0)
estimator.fit(X, y)

SVC(random_state=0)

We inspect the learned features.

In [5]:
estimator.support_vectors_[:4]

array([[-0.77244139,  0.3607576 , -2.38110133,  0.08757   ],
       [ 1.14946035,  0.62254594,  0.37302939,  0.45965795],
       [-0.77694695,  0.31434299, -2.26231851,  0.06339125],
       [ 0.79010037,  0.68530624, -0.44740487,  0.44692959]])

And check the accuracy.

In [6]:
estimator.score(X, y)

0.905

*A quick ML refresher:*
- *There are [3 different approaches](https://scikit-learn.org/0.15/modules/model_evaluation.html) to evaluate the quality of predictions of a model. One of them is the **estimator score method**. Estimators have a score method providing a default evaluation criterion for the problem they are designed to solve, which is discussed in each estimator's documentation.*

### Hyperparameters

All estimators have parameters (often called *hyperparameters*). They affect the fit, but are specified up front instead of learned during training.

In [7]:
estimator = SVC(C=0.00001, shrinking=False, random_state=0)
estimator.fit(X, y)

SVC(C=1e-05, random_state=0, shrinking=False)

And the score of this hyperparameter set for this model:

In [8]:
estimator.score(X, y)

0.5007

### Hyperparameter Optimization

There are a few ways to learn the best *hyper*parameters while training. One is `GridSearchCV`.
As the name implies, this does a brute-force search over a grid of hyperparameter combinations.

*A quick [ML refresher](https://scikit-learn.org/stable/getting_started.html):*
- *Scikit-learn provides tools to automatically find the best parameter combinations via cross-validation (which is the "CV" in `GridSearchCV`).*

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

# Brute-force search over a grid of hyperparameter combinations
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END ................................C=0.001, kernel=rbf; total time=   7.3s
[CV] END ................................C=0.001, kernel=rbf; total time=   7.2s
[CV] END ...............................C=0.001, kernel=poly; total time=   2.5s
[CV] END ...............................C=0.001, kernel=poly; total time=   2.5s
[CV] END .................................C=10.0, kernel=rbf; total time=   1.9s
[CV] END .................................C=10.0, kernel=rbf; total time=   1.8s
[CV] END ................................C=10.0, kernel=poly; total time=   2.4s
[CV] END ................................C=10.0, kernel=poly; total time=   2.2s
CPU times: user 33.4 s, sys: 230 ms, total: 33.6 s
Wall time: 33.6 s


GridSearchCV(cv=2,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             param_grid={'C': [0.001, 10.0], 'kernel': ['rbf', 'poly']},
             verbose=2)

In [11]:
grid_search.best_params_, grid_search.best_score_

({'C': 10.0, 'kernel': 'rbf'}, 0.9086000000000001)

**Section recap:** We have

* segmented ML scaling problems into two categories.
  * CPU-bound problems where scaling up the model size is the issue
  * RAM-bound problems where scaling up the data size is the issue
* carried out a typical Scikit-Learn workflow for ML problems with small model(s) and small data.
* reviewed hyperparameters and hyperparameter optimization in Scikit-Learn.

## 1. Compute Bound: Single-machine parallelism with Joblib

<img src="images/joblib_logo.svg" alt="Joblib logo" style="width: 300px;"/>

In this section we'll see how [Joblib](https://joblib.readthedocs.io/en/latest/) ("*a set of tools to provide lightweight pipelining in Python*") gives us parallelism on our laptop. Here's what our grid search graph would look like if we set up six training "jobs" in parallel:

![](images/unmerged_grid_search_graph.svg)

With Joblib, we can say that Scikit-Learn has *single-machine* parallelism.
Any Scikit-Learn estimator that can operate in parallel exposes an `n_jobs` keyword, which tells you how many tasks to run in parallel. Specifying `-1` jobs means running the maximum possible number of tasks in parallel.

In [12]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
CPU times: user 5.72 s, sys: 182 ms, total: 5.91 s
Wall time: 15.2 s


GridSearchCV(cv=2,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             n_jobs=-1,
             param_grid={'C': [0.001, 10.0], 'kernel': ['rbf', 'poly']},
             verbose=2)

In [13]:
grid_search.best_params_, grid_search.best_score_

({'C': 10.0, 'kernel': 'rbf'}, 0.9086000000000001)

**Section recap:** Previously, we

* solved an ML problem with a single machine (with Scikit-Learn).

In this section, we

* solved an ML problem with a single machine and parallelism (with Scikit-Learn and Joblib).

## 2. Compute Bound: Multi-machine parallelism with Dask

<img src="images/dask_horizontal_no_pad.svg" alt="Dask logo" style="width: 500px;"/>

In this section we'll see how Dask (plus Joblib and Scikit-Learn) gives us multi-machine parallelism. Here's what our grid search graph would look like if we allowed Dask to schedule our training "jobs" over multiple machines in our cluster:

![](images/merged_grid_search_graph.svg)

We can say that Dask can talk to Scikit-Learn (via Joblib) so that our *cluster* is used to train a model. 

If we run this on a laptop, it will take quite some time, but the CPU usage will be satisfyingly near 100% for the duration. To run faster, we would need a distributed cluster. That would mean putting something in the call to `Client` something like

```
c = Client('tcp://my.scheduler.address:8786')
```

Details on the many ways to create a cluster can be found [here](https://docs.dask.org/en/latest/setup/single-distributed.html).

Let's try it on a larger problem (more hyperparameters). First we'll instantiante a Dask Client.

In [14]:
import joblib
import dask.distributed

client = dask.distributed.Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55672,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://192.168.1.14:55683,Total threads: 3
Dashboard: http://192.168.1.14:55686/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:55677,


Here we expand our problem by specifying more hyperparameters before training.

In [15]:
param_grid = {
    'C': [0.001, 0.1, 1.0, 2.5, 5, 10.0],
    # Uncomment this for larger Grid searches on a cluster
    'kernel': ['rbf', 'poly', 'linear'],
    'shrinking': [True, False],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)

Now, watch this. We can fit our estimator with multi-machine parallelism by quickly *switching to a Dask parallel backend*.

In [16]:
%%time
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
CPU times: user 8.35 s, sys: 682 ms, total: 9.03 s
Wall time: 48.9 s


How does this work so seamlessly? Dask-ML developers worked with the Scikit-Learn and Joblib developers to implement a Dask parallel backend. So internally, scikit-learn now talks to Joblib, and Joblib talks to Dask, and Dask is what handles scheduling all of those tasks on multiple machines.

The best parameters and best score:

In [17]:
grid_search.best_params_, grid_search.best_score_

({'C': 10.0, 'kernel': 'rbf', 'shrinking': True}, 0.9086000000000001)

**Section recap:** Previously, we

* solved an ML problem with a single machine (with Scikit-Learn).
* solved an ML problem with a single machine and parallelism (with Scikit-Learn and Joblib).

In this section, we

* solved an ML problem with multiple machines and parallelism (with Scikit-Learn, Joblib and Dask).

## 3. Memory Bound: Single/Multi machine parallelism with Dask-ML

We have seen how to work with larger models, but sometimes you'll want to train on a larger than memory dataset. `dask-ml` has implemented estimators that work well on Dask Arrays and DataFrames that may be larger than your machine's RAM.

In [18]:
import dask.array as da
import dask.delayed
from sklearn.datasets import make_blobs
import numpy as np

We'll make a small (random) dataset locally using Scikit-Learn.

In [19]:
n_centers = 12
n_features = 20

X_small, y_small = make_blobs(n_samples=1000, centers=n_centers, n_features=n_features, random_state=0)

centers = np.zeros((n_centers, n_features))

for i in range(n_centers):
    centers[i] = X_small[y_small == i].mean(0)
    
centers[:4]

array([[ 1.00796679,  4.34582168,  2.15175661,  1.04337835, -1.82115164,
         2.81149666, -1.18757701,  7.74628882,  9.36761449, -2.20570731,
         5.71142324,  0.41084221,  1.34168817,  8.4568751 , -8.59042755,
        -8.35194302, -9.55383028,  6.68605157,  5.34481483,  7.35044606],
       [ 9.49283024,  6.1422784 , -0.97484846,  5.8604399 , -7.61126963,
         2.86555735, -7.25390288,  8.89609285,  0.33510318, -1.79181328,
        -4.66192239,  5.43323887, -0.86162507,  1.3705568 , -9.7904172 ,
         2.3613231 ,  2.20516237,  2.20604823,  8.76464833,  3.47795068],
       [-2.67206588, -1.30103177,  3.98418492, -8.88040428,  3.27735964,
         3.51616445, -5.81395151, -7.42287114, -3.73476887, -2.89520363,
         1.49435043, -1.35811028,  9.91250767, -7.86133474, -5.78975793,
        -6.54897163,  3.08083281, -5.18975209, -0.85563107, -5.06615534],
       [-6.85980599, -7.87144648,  3.33572279, -7.00394241, -5.97224874,
        -2.55638942,  6.36329802, -7.97988653,  

The small dataset will be the template for our large random dataset.
We'll use `dask.delayed` to adapt `sklearn.datasets.make_blobs`, so that the actual dataset is being generated on our workers. 

In [20]:
n_samples_per_block = 200000
n_blocks = 500

delayeds = [dask.delayed(make_blobs)(n_samples=n_samples_per_block,
                                     centers=centers,
                                     n_features=n_features,
                                     random_state=i)[0]
            for i in range(n_blocks)]
arrays = [da.from_delayed(obj, shape=(n_samples_per_block, n_features), dtype=X.dtype)
          for obj in delayeds]
X = da.concatenate(arrays)
X

dask.array<concatenate, shape=(100000000, 20), dtype=float64, chunksize=(200000, 20), chunktype=numpy.ndarray>

In [21]:
X = X.persist()  # Only run this on the cluster.

The algorithms implemented in Dask-ML are scalable. They handle larger-than-memory datasets just fine.

They follow the scikit-learn API, so if you're familiar with Scikit-Learn, you'll feel at home with Dask-ML.

In [22]:
from dask_ml.cluster import KMeans

In [23]:
clf = KMeans(init_max_iter=3, oversampling_factor=10)

In [24]:
%time clf.fit(X)

[CV] END .................................C=10.0, kernel=rbf; total time=   2.5s
[CV] END .................................C=10.0, kernel=rbf; total time=   2.6s
[CV] END ................................C=10.0, kernel=poly; total time=   3.0s
[CV] END ................................C=10.0, kernel=poly; total time=   3.1s
[CV] END ...............................C=0.001, kernel=poly; total time=   3.4s
[CV] END ...............................C=0.001, kernel=poly; total time=   3.5s
[CV] END ................................C=0.001, kernel=rbf; total time=   8.1s
[CV] END ................................C=0.001, kernel=rbf; total time=   8.2s
CPU times: user 1min 29s, sys: 21.6 s, total: 1min 51s
Wall time: 7min 32s


KMeans(init_max_iter=3, oversampling_factor=10)

In [25]:
clf.labels_

dask.array<astype, shape=(100000000,), dtype=int32, chunksize=(200000,), chunktype=numpy.ndarray>

*A quick [ML refresher](https://datascience.stackexchange.com/questions/44108/difference-between-a-target-and-a-label-in-machine-learning):*
- *A label is a true outcome of the target variable `y`. In supervised learning, the target labels are known for the trainining dataset but not for the test.*

In [26]:
clf.labels_[:10].compute()

array([2, 1, 2, 0, 0, 0, 5, 1, 1, 3], dtype=int32)

In [28]:
client.close()

**Section recap:** Previously, we

* solved an ML problem with a single machine (with Scikit-Learn).
* solved an ML problem with a single machine and parallelism (with Scikit-Learn and Joblib).
* solved an ML problem with multiple machines and parallelism (with Scikit-Learn, Joblib and Dask).

In this section, we

* solved an ML problem with larger-than-memory data using multiple machines and parallelism (Dask-ML).

## 4. Multi-machine parallelism in the cloud with Coiled

<br>
<img src="images/Coiled-Logo_Horizontal_RGB_Black.png" alt="Coiled logo" width=25%/>
<br>

In this section we'll see how Coiled allows us to solve machine learning problems with multi-machine parallelism in the cloud.

Coiled, [among other things](https://coiled.io/product/), provides hosted and scalable Dask clusters. The biggest barriers to entry for doing machine learning at scale are "Do you have access to a cluster?" and "Do you know how to manage it?" Coiled solves both of those problems. Let's see how.

We'll spin up a Coiled cluster (with 10 workers in this case), then instantiante a Dask Client to use with that cluster.

In [ ]:
import coiled
from dask.distributed import Client

In [ ]:
# Spin up a Coiled cluster, instantiate a Client
cluster = coiled.Cluster(n_workers=10, software="pavithraes/data-science-at-scale",)
client = Client(cluster)
client

### Compute bound: Dask parallel backend

Again, we can fit our estimator with multi-machine paralellism by quickly switching to a Dask parallel backend. This time, this multi-machine parallelism is *in the cloud* because we've set up our Dask clusters via Coiled.

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=4, random_state=0)

In [ ]:
%%time
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

Our cluster being a cloud-based cluster that adds no complexity is Coiled's mission on full display.

The best parameters and best score:

In [ ]:
grid_search.best_params_, grid_search.best_score_

### Memory bound: Dask-ML

Similarly, we can use Dask-ML estimators on the cloud to work with larger datasets.

In [ ]:
n_centers = 12
n_features = 20

X_small, y_small = make_blobs(n_samples=1000, centers=n_centers, n_features=n_features, random_state=0)

centers = np.zeros((n_centers, n_features))

for i in range(n_centers):
    centers[i] = X_small[y_small == i].mean(0)


In [ ]:
n_samples_per_block = 200000
n_blocks = 500

delayeds = [dask.delayed(make_blobs)(n_samples=n_samples_per_block,
                                     centers=centers,
                                     n_features=n_features,
                                     random_state=i)[0]
            for i in range(n_blocks)]
arrays = [da.from_delayed(obj, shape=(n_samples_per_block, n_features), dtype=X.dtype)
          for obj in delayeds]
X = da.concatenate(arrays)
X = X.persist()

In [ ]:
from dask_ml.cluster import KMeans

In [ ]:
clf = KMeans(init_max_iter=3, oversampling_factor=10)

In [ ]:
%time clf.fit(X)

Computng the labels:

In [ ]:
clf.labels_[:10].compute()

In [ ]:
client.close()

**Section recap:** Previously, we

* solved an ML problem with a single machine (with Scikit-Learn).
* solved an ML problem with a single machine and parallelism (with Scikit-Learn and Joblib).
* solved an ML problem with a multiple machines and parallelism (with Scikit-Learn, Joblib, and Dask).
* solved an ML problem with larger-than-memory data using multiple machines and parallelism (Dask-ML).

In this section, we

* solved two ML problem with multiple machines *in the cloud* and parallelism (with Scikit-Learn, Joblib, Dask, and Coiled).

## Recap (for this entire notebook!)

We did the following:

* broke down machine learning scaling problems into to two categories (data size vs. model size).
* solved an ML problem with a single machine (with Scikit-Learn).
* solved an ML problem with a single machine and parallelism (with Scikit-Learn and Joblib).
* solved an ML problem with a multiple machines and parallelism (with Scikit-Learn, Joblib and Dask).
* used `dask-ml` estimators that work well on Dask Arrays and DataFrames to train on datasets larger than your machine's RAM.
* solved an ML problem with a multiple machines *in the cloud* and parallelism (with Scikit-Learn, Joblib, Dask and Coiled).